In [1]:
print("""
@File         : CH13.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-08-31 10:07:21
@Email        : cuixuanstephen@gmail.com
@Description  : 处理 Excel 电子表格
""")


@File         : CH13.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-08-31 10:07:21
@Email        : cuixuanstephen@gmail.com
@Description  : 处理 Excel 电子表格



In [2]:
%cd ../

d:\Python-3\ABSP2E


In [3]:
import openpyxl

## 读取 Excel 文档

### 用 openpyxl 模块打开 Excel 文档

`openpyxl.load_workbook()` 函数接收文件名，并返回一个 `Workbook` 数据类型的值。这个 `Workbook` 对象代表这个 Excel 文件，这有点儿类似于 `File` 对象代表一个打开的文本文件。

In [4]:
wb = openpyxl.load_workbook('sources/example.xlsx')
type(wb)

openpyxl.workbook.workbook.Workbook

### 从工作簿中取得工作表

访问 `sheetnames` 属性可以取得工作簿中所有表名的列表。

In [5]:
wb.sheetnames

['Sheet1', 'Sheet2', 'Sheet3']

In [6]:
sheet = wb['Sheet3']
sheet

<Worksheet "Sheet3">

In [7]:
type(sheet)

openpyxl.worksheet.worksheet.Worksheet

In [8]:
sheet.title

'Sheet3'

In [9]:
another_sheet = wb.active
another_sheet

<Worksheet "Sheet1">

每个表由一个 `Worksheet` 对象表示，取得它的方法是使用带方括号的工作表名称字符串，这和取得字典的键一样。最后，可以使用 `Workbook` 对象的 `active` 属性来取得工作簿的活动表。在取得 `Worksheet` 对象后，可以通过 `title` 属性取得它的名称。

### 从表中取得单元格

有了 `Worksheet` 对象后，就可以按名字访问 `Cell` 对象。

In [10]:
sheet = wb['Sheet1']
sheet['A1'].value

datetime.datetime(2015, 4, 5, 13, 34, 2)

In [11]:
c = sheet['B1']
c.value

'Apples'

In [12]:
f'Row {c.row}, Column {c.column} is {c.value}'

'Row 1, Column 2 is Apples'

In [13]:
f'Cell {c.coordinate} is {c.value}'

'Cell B1 is Apples'

In [14]:
sheet['C1'].value

73

`Cell` 对象有一个 value 属性，它包含这个单元格中保存的值。`Cell` 对象也有 `row`、`column` 和 `coordinate` 属性，可以提供该单元格的位置信息。

openpyxl 将自动解释列中的日期，将它们返回为 `datetime` 值，而不是字符串。

用字母来指定列，这在程序中可能有点儿奇怪，特别是在 Z 列之后，列开始使用两个字母：AA、AB、AC 等表示。作为替代，在调用表的 `cell()` 方法时，传入整数作为该方法的 `row` 和 `column` 关键字参数，这样也可以得到一个单元格。**第一行或第一列的整数是 1，不是 0。**

In [15]:
sheet.cell(row=1, column=2)

<Cell 'Sheet1'.B1>

In [16]:
sheet.cell(row=1, column=2).value

'Apples'

In [17]:
for i in range(1, 8, 2):
    print(i, sheet.cell(row=i, column=2).value)

1 Apples
3 Pears
5 Apples
7 Strawberries


可以通过 `Worksheet` 对象的 `max_row` 和 `max_column` 属性来确定表的大小。

In [18]:
sheet = wb['Sheet1']
sheet.max_row # Get the highest row number.

7

In [19]:
sheet.max_column

3

请注意，`max_column` 属性是一个整数。

### 列字母和数字之间的转换
要从字母转换到数字，就调用 `openpyxl.utils.column_index_from_string()` 函数。要从数字转换到字母，就调用 `openpyxl.utils.get_column_letter()` 函数。

In [20]:
from openpyxl.utils import get_column_letter, column_index_from_string

In [21]:
get_column_letter(1)

'A'

In [22]:
get_column_letter(2)

'B'

In [23]:
get_column_letter(27)

'AA'

In [24]:
get_column_letter(900)

'AHP'

In [25]:
wb = openpyxl.load_workbook('sources/example.xlsx')
sheet = wb['Sheet1']
get_column_letter(sheet.max_column)

'C'

In [26]:
column_index_from_string("A")

1

In [27]:
column_index_from_string("AA")

27

### 从表中取得行和列

可以将 `Worksheet` 对象切片，取得电子表格中一行、一列或一个矩形区域中的所有 `Cell` 对象。然后可以循环遍历这个切片中的所有单元格。

In [29]:
sheet = wb['Sheet1']
tuple(sheet['A1': 'C3']) # Get all cells from A1 to C3.

((<Cell 'Sheet1'.A1>, <Cell 'Sheet1'.B1>, <Cell 'Sheet1'.C1>),
 (<Cell 'Sheet1'.A2>, <Cell 'Sheet1'.B2>, <Cell 'Sheet1'.C2>),
 (<Cell 'Sheet1'.A3>, <Cell 'Sheet1'.B3>, <Cell 'Sheet1'.C3>))

In [30]:
for row in sheet['A1': 'C3']:
    for cell in row:
        print(cell.coordinate, cell.value)
    print('--- END OF ROW ---')

A1 2015-04-05 13:34:02
B1 Apples
C1 73
--- END OF ROW ---
A2 2015-04-05 03:41:23
B2 Cherries
C2 85
--- END OF ROW ---
A3 2015-04-06 12:46:51
B3 Pears
C3 14
--- END OF ROW ---


这里，我们指明需要从 A1 到 C3 的矩形区域中的 `Cell` 对象，我们还得到了一个 `Generator` 对象，它包含该区域中的 `Cell` 对象。为了弄清楚这个 `Generator` 对象，可以对它使用 `tuple()` 方法以在一个元组中列出它的 `Cell` 对象。

要访问特定行或列的单元格的值，也可以利用 `Worksheet` 对象的 `rows` 和 `columns` 属性。这些属性必须被 `list()` 函数转换为列表，才能使用方括号和索引。

In [32]:
sheet = wb.active
list(sheet.columns)[1]

(<Cell 'Sheet1'.B1>,
 <Cell 'Sheet1'.B2>,
 <Cell 'Sheet1'.B3>,
 <Cell 'Sheet1'.B4>,
 <Cell 'Sheet1'.B5>,
 <Cell 'Sheet1'.B6>,
 <Cell 'Sheet1'.B7>)

In [33]:
for cell in list(sheet.columns)[1]:
    print(cell.value)

Apples
Cherries
Pears
Oranges
Apples
Bananas
Strawberries


利用 `Worksheet` 对象的 `rows` 属性，可以得到一个元组构成的元组。内部的每个元组都代表 1 行，包含该行中的 `Cell` 对象。`columns` 属性也会给你一个元组构成的元组，内部的每个元组都包含 1 列中的 `Cell` 对象。

### 工作簿、工作表、单元格

下面是从电子表格文件中读取单元格涉及的所有函数、方法和数据类型。
1. 导入 `openpyxl` 模块。
2. 调用 `openpyxl.load_workbook()` 函数。
3. 取得 `Workbook` 对象。
4. 使用 `active` 或 `sheetnames` 属性。
5. 取得 `Worksheet` 对象。
6. 使用索引或工作表的 `cell()` 方法，带上 `row` 和 `column` 关键字参数。
7. 取得 `Cell` 对象。
8. 读取 `Cell` 对象的 `value` 属性。

## [项目：从电子表格中读取数据](https://github.com/JPL-JUNO/Python-3/blob/main/ABSP2E/sources/readCensusExcel.py)

## 写入 `Excel` 文档

### 创建并保存 Excel 文档

调用 `openpyxl.Workbook()` 函数以创建一个新的空 `Workbook` 对象。

In [34]:
import openpyxl
wb = openpyxl.Workbook() # Create a blank workbook
wb.sheetnames

['Sheet']

In [35]:
sheet = wb.active
sheet.title

'Sheet'

In [36]:
sheet.title = 'Spam Bacon Eggs Sheet'
wb.sheetnames

['Spam Bacon Eggs Sheet']

当修改 `Workbook` 对象或它的工作表和单元格时，电子表格文件不会保存，除非你调用 `save()` 工作簿方法。

In [37]:
wb.save('data/example_copy.xlsx') # Save the workbook

**当你编辑从文件中加载的一个电子表格时，总是应该将新的、编辑过的电子表格保存到不同的文件名中。这样，如果代码有 bug，导致新的保存到文件中的数据不对，那么还有最初的电子表格文件可以处理。**

### 创建和删除工作表

利用 `create_sheet()` 方法和 `del 操作符可以在工作簿中添加或删除工作表。

In [38]:
wb = openpyxl.Workbook()
wb.sheetnames

['Sheet']

In [39]:
wb.create_sheet()

<Worksheet "Sheet1">

In [40]:
wb.sheetnames

['Sheet', 'Sheet1']

In [41]:
wb.create_sheet(index=0, title='Middle Sheet')
wb.sheetnames

['Middle Sheet', 'Sheet', 'Sheet1']

In [42]:
wb.create_sheet(index=2, title='First Sheet')

<Worksheet "First Sheet">

In [43]:
wb.sheetnames

['Middle Sheet', 'Sheet', 'First Sheet', 'Sheet1']

`create_sheet()` 方法返回一个新的 `Worksheet` 对象，其名为 `SheetX`，它默认是工作簿的最后一个工作表。或者，可以利用 `index` 和 `title` 关键字参数指定新工作表的索引和名称。

In [44]:
wb.sheetnames

['Middle Sheet', 'Sheet', 'First Sheet', 'Sheet1']

In [45]:
del wb['Middle Sheet']

In [46]:
del wb['Sheet1']

In [47]:
wb.sheetnames

['Sheet', 'First Sheet']

可以使用 `del` 操作符，从工作簿中删除一个工作表，就像用它从字典中删除一个键-值对一样。

### 将值写入单元格

将值写入单元格，很像将值写入字典中的键。

In [48]:
wb = openpyxl.Workbook()
sheet = wb['Sheet']
sheet['A1'] = 'Hello, world!'
sheet['A1'].value

'Hello, world!'

## [项目：更新电子表格]()

![产品销售的电子表格](../img/produceSales.png)

列分别是销售产品的类型（A）、产品每磅的价格（B）、销售的磅数（C），以及这次销售的总收入（D）。TOTAL 列设置为Excel 公式=ROUND(B3*C3, 2)，它将每磅的成本乘以销售的磅数，并将结果取整到分。有了这个公式，如果列 B 或 C 发生变化，TOTAL 列中的单元格将自动更新。

## 设置单元格的字体风格

为了定义单元格的字体风格，需要从 `openpyxl.styles` 模块导入 `Font()` 函数：

In [50]:
from openpyxl.styles import Font
wb = openpyxl.Workbook()
sheet = wb['Sheet']
italic_24_font = Font(size=24, italic=True) # Create a font
sheet['A1'].font = italic_24_font
sheet['A1'] = 'Hello, openpyxl'
wb.save('data/styles.xlsx')

## `Font` 对象
要设置 `font` 属性，就向 `Font()` 函数传入关键字参数。

|关键字参数|数据类型|描述|
|---|---|---|
|`name`|字符串|字体名称，如 'Calibri' 或 'Times New Roman'|
|`size`|整型|大小点数|
|`bold`|布尔型|`True` 表示粗体|
|`italic`|布尔型|`True` 表示斜体|

In [52]:
wb = openpyxl.Workbook()
sheet = wb['Sheet']

font_obj_1 = Font(name='Consols', bold=True)
sheet['A1'].font = font_obj_1
sheet['A1'] = 'Bold Consols'

font_obj_2 = Font(size=24, italic=True)
sheet['B3'] = '24 pt Italic'
sheet['B3'].font = font_obj_2
wb.save('data/styles.xlsx')

## 公式
公式以一个等号开始，可以配置单元格来让它包含通过其他单元格计算得到的值。将利用 `openpyxl` 模块，用编程的方式在单元格中添加公式，就像添加普通的值一样。

In [53]:
wb = openpyxl.Workbook()
sheet = wb.active
sheet['A1'] = 200
sheet['A2'] = 300
sheet['A3'] = '=SUM(A1:A2)'
wb.save('data/write_formula.xlsx')

## 调整行和列

### 设置行高和列宽
`Worksheet` 对象有 `row_dimensions` 和 `column_dimensions` 属性，分别用于控制行高和列宽。

In [54]:
wb = openpyxl.Workbook()
sheet = wb.active
sheet['A1'] = 'Tall row'
sheet['B2'] = 'Wide Column'

sheet.row_dimensions[1].height = 70
sheet.column_dimensions['B'].width = 20
wb.save('data/dimensions.xlsx')

工作表的 `row_dimensions` 和 `column_dimensions` 是像字典一样的值，`row_dimensions` 包含 `RowDimension` 对象，`column_dimensions` 包含 `ColumnDimension` 对象。在 `row_dimensions` 中，可以用行的编号来访问一个对象。在 `column_dimensions` 中，可以用列的字母来访问一个对象。

一旦有了 `RowDimension` 对象，就可以设置它的高度。一旦有了 `ColumnDimension` 对象，就可以设置它的宽度。

### 合并和拆分单元格
利用 `merge_cells()` 工作表方法，可以将一个矩形区域中的单元格合并为一个单元格。

In [57]:
import openpyxl


wb = openpyxl.Workbook()
sheet = wb.active
sheet.merge_cells('A1:D3')
sheet['A1'] = 'Twelve cells merged together'
sheet.merge_cells('C5:D5')
sheet['C5'] = 'Two merged cells.'
wb.save('data/merged.xlsx')

`merge_cells()` 的参数是一个字符串，表示要合并的矩形区域左上角和右下角的单元格：'A1:D3' 将 12 个单元格合并为一个单元格。要设置合并后单元格的值，只需要设置这一组合并单元格左上角的单元格的值。

要拆分单元格，就调用 `unmerge_ cells()` 工作表方法。

In [58]:
wb = openpyxl.load_workbook('data/merged.xlsx')
sheet = wb.active
sheet.unmerge_cells('A1:D3')
sheet.unmerge_cells('C5:D5')
wb.save('data/merged.xlsx')

### 冻结窗格
对于太大而不能一屏显示的电子表格，“冻结”顶部的几行或最左边的几列是很有帮助的。例如，就算用户滚动电子表格，冻结的列或行表头也是始终可见的。这称为“冻结窗格”。在 `openpyxl` 中，每个 `Worksheet` 对象都有一个 `freeze_panes` 属性，该属性可以设置为一个 `Cell` 对象或一个单元格坐标的字符串。

>
> **请注意，单元格上边的所有行和左边的所有列都会冻结，但单元格所在的行和列不会冻结。** 这一点与 Excel 中的操作是一致的。

要解冻所有的单元格，就将 `freeze_panes` 设置为 `None` 或 'A1'。

In [59]:
wb = openpyxl.load_workbook('data/produceSales.xlsx')
sheet = wb.active
sheet.freeze_panes = 'A2'
wb.save('res/freezeExample.xlsx')

## 图表
openpyxl 支持利用工作表中单元格的数据来创建条形图、折线图、散点图和饼图。

要创建图表，需要做下列事情：

1. 从一个矩形区域选择单元格来创建一个 `Reference` 对象。
2. 通过传入 `Reference` 对象来创建一个 `Series` 对象。
3. 创建一个 `Chart` 对象。
4. 将 `Series` 对象添加到 `Chart` 对象。
5. 可选地设置 `Chart` 对象的 `drawing.top`、`drawing.left`、`drawing.width` 和 `drawing.height` 属性。
6. 将 `Chart` 对象添加到 `Worksheet` 对象。

`Reference` 对象是通过调用 `openpyxl.charts. Reference()` 函数并传入以下 3 个参数创建的。
1. 包含图表数据的 `Worksheet` 对象。
2. 两个整数的元组，代表矩形选择区域的左上角单元格，该区域包含图表数据：元组中第一个整数是行，第二个整数是列。请注意第一行是 1，不是 0。
3. 两个整数的元组，代表矩形选择区域的右下角单元格，该区域包含图表数据：元组中第一个整数是行，第二个整数是列。

> 感觉是两个坐标的四个值。

In [62]:
import openpyxl.chart
import openpyxl.chart.series


wb = openpyxl.Workbook()
sheet = wb.active
for i in range(1, 11):
    sheet[f'A{i}'] = i
    
ref_obj = openpyxl.chart.Reference(sheet, min_col=1, min_row=1, max_col=1, max_row=10)
ser_obj = openpyxl.chart.Series(ref_obj, title='First Series')

chart_obj = openpyxl.chart.BarChart()
chart_obj.title = 'My Chart'
chart_obj.append(ser_obj)
# 图片的左上角将位于 C5
sheet.add_chart(chart_obj, 'C5')
wb.save('res/sample_chart.xlsx')